### Auto Arima

In [4]:
import sys
import os
app = os.path.abspath(os.path.join(os.getcwd(), os.pardir,'app'))
sys.path.append(app)
from utils import flights
from sklearn.model_selection import train_test_split
import pandas as pd
import itertools

f = flights('LGAV')
dfs = f.get_dfs()
all_summaries=dict()
times = list()

for df_ , differencing in itertools.product(dfs, [True, False]):
    title = df_.name
    df = df_.copy()
    
    train, test = train_test_split(df, test_size=60, shuffle=False)
    
    arimaResults, elapsed_time = f.arima(train, test, differencing)
    times.append(elapsed_time)

    all_summaries[title+", Differencing :" + str(differencing)] = arimaResults[1]
    f.save_results(test[f.airport].values , arimaResults[0].values.round(), title, differencing)

In [2]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir,'results')))
import pickle 
with open('summaries.pickle', 'wb') as file:
  # Use the pickle.dump() method to save the dictionary
  pickle.dump(all_summaries, file)

# import pickle
# # Open the file for reading in binary mode
# with open('summaries.pickle', 'rb') as f:
#   # Use the pickle.load() method to load the data
#   my_dict = pickle.load(f)

In [3]:
f.results['Time']=times
f.results.to_csv('Arima.csv')
f.results

RMSE  MAPE   MAE  Differencing         Time
Model Features                                                             
Arima No exogenous             81.24  0.39 51.70          True  0:00:44.425
      No exogenous             79.76  0.45 53.83         False  0:01:05.483
      Time as exogenous        82.37  0.39 52.18          True  0:07:07.864
      Time as exogenous        79.55  0.51 56.65         False  0:17:42.342
      Quarantines as exogenous 81.24  0.39 51.70          True  0:01:34.822
      Quarantines as exogenous 82.51  0.36 50.83         False  0:02:07.827
      Covid data as exog       81.34  0.39 51.73          True  0:01:06.410
      Covid data as exog       82.37  0.37 51.32         False  0:01:15.872

---
### Neural Networks

In [1]:
import sys
import os
app = os.path.abspath(os.path.join(os.getcwd(), os.pardir,'app'))
sys.path.append(app)
from utils import flights
import pandas as pd
import itertools

f = flights('LGAV')
dfs = f.get_dfs()
times = list()

for df_ , differencing in itertools.product(dfs, [True, False]):
    title = df_.name
    df=df_.copy()
    
    if differencing:
        if title != 'Covid data as exog':
            df[f.airport] = df[f.airport].diff()
        else:
            df = df.diff()

    x_train, y_train, x_test, y_test = f.add_lags(df, past=7, test_size=60)
    
    mlpResults, elapsed_time = f.mlp(x_train, y_train, x_test, (14,7))
    times.append(elapsed_time)

    if differencing:
        mlpResults = f.invert_predictions(mlpResults)
    

    f.save_results(f.truth, mlpResults, title, differencing)

    lstmResults, elapsed_time = f.lstm(x_train, y_train, x_test, (14,7))
    times.append(elapsed_time)

    if differencing:
        lstmResults = f.invert_predictions(lstmResults)
    
    f.save_results(f.truth, lstmResults, title, differencing)

    tcnResults, elapsed_time = f.tcn(x_train, y_train, x_test, (30,7))
    times.append(elapsed_time)

    if differencing:
        tcnResults = f.invert_predictions(tcnResults)
    
    f.save_results(f.truth, tcnResults, title, differencing)

In [2]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir,'results')))
f.results['Time']=times
nnResults = f.results.sort_index()
nnResults.to_csv('NeuralNetworks.csv')
nnResults

RMSE  MAPE    MAE  Differencing  \
Model        Features                                                     
LSTM         Covid data as exog       125.17  1.54 110.87          True   
             Covid data as exog        96.79  0.39  61.80         False   
             No exogenous              83.71  0.38  53.93          True   
             No exogenous              49.03  0.34  32.20         False   
             Quarantines as exogenous  84.22  0.82  72.83          True   
             Quarantines as exogenous  55.52  0.49  39.70         False   
             Time as exogenous        101.76  0.62  75.52          True   
             Time as exogenous         47.87  0.36  33.32         False   
MLPRegressor Covid data as exog        81.60  0.76  68.55          True   
             Covid data as exog       135.44  1.00 112.30         False   
             No exogenous              93.75  0.38  57.77          True   
             No exogenous              44.49  0.31  30.22         False   
             Quarantines as exogenous  85.69  0.66  64.97          True   
             Quarantines as exogenous  44.48  0.28  28.85         False   
             Time as exogenous         77.49  0.55  57.72          True   
             Time as exogenous         49.52  0.34  34.32         False   
TCN          Covid data as exog       137.13  1.26 111.38          True   
             Covid data as exog        47.20  0.38  33.82         False   
             No exogenous             235.96  2.95 192.80          True   
             No exogenous              48.80  0.32  33.37         False   
             Quarantines as exogenous  93.41  0.50  65.92          True   
             Quarantines as exogenous  47.81  0.38  31.08         False   
             Time as exogenous        154.75  1.89 129.35          True   
             Time as exogenous         47.29  0.32  30.63         False   

                                              Time  
Model        Features                               
LSTM         Covid data as exog        0:00:05.338  
             Covid data as exog        0:00:09.000  
             No exogenous              0:00:07.257  
             No exogenous              0:00:05.730  
             Quarantines as exogenous  0:00:06.780  
             Quarantines as exogenous  0:00:09.679  
             Time as exogenous         0:00:09.060  
             Time as exogenous         0:00:06.753  
MLPRegressor Covid data as exog        0:00:00.085  
             Covid data as exog        0:00:00.168  
             No exogenous              0:00:00.042  
             No exogenous              0:00:00.091  
             Quarantines as exogenous  0:00:00.071  
             Quarantines as exogenous  0:00:00.095  
             Time as exogenous         0:00:00.041  
             Time as exogenous         0:00:00.237  
TCN          Covid data as exog        0:01:28.562  
             Covid data as exog        0:00:16.276  
             No exogenous              0:00:55.868  
             No exogenous              0:00:22.782  
             Quarantines as exogenous  0:01:06.691  
             Quarantines as exogenous  0:00:19.738  
             Time as exogenous         0:01:42.322  
             Time as exogenous         0:00:29.911

---
### Ensembles

In [1]:
import sys
import os
app = os.path.abspath(os.path.join(os.getcwd(), os.pardir,'app'))
sys.path.append(app)
from utils import flights
import pandas as pd
import itertools

f = flights('LGAV')
dfs = f.get_dfs()
times = list()

for df_ , differencing in itertools.product(dfs, [True, False]):
    title = df_.name
    df=df_.copy()
    
    if differencing:
        if title != 'Covid data as exog':
            df[f.airport] = df[f.airport].diff()
        else:
            df = df.diff()

    x_train, y_train, x_test, y_test = f.add_lags(df, past=7, test_size=60)
    
    xgboostResults, elapsed_time = f.xgboost(x_train, y_train, x_test)
    times.append(elapsed_time)

    if differencing:
        xgboostResults = f.invert_predictions(xgboostResults)

    f.save_results(f.truth, xgboostResults, title, differencing)

    hgboostResults, elapsed_time = f.hgboost(x_train, y_train, x_test)
    times.append(elapsed_time)

    if differencing:
        hgboostResults = f.invert_predictions(hgboostResults)
    
    f.save_results(f.truth, hgboostResults, title, differencing)

    adaboostResults, elapsed_time = f.adaboost(x_train, y_train, x_test)
    times.append(elapsed_time)

    if differencing:
        adaboostResults = f.invert_predictions(adaboostResults)
    
    f.save_results(f.truth, adaboostResults, title, differencing)

In [2]:
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir,'results')))
f.results['Time']=times
ensemlbesResults = f.results.sort_index()
ensemlbesResults.to_csv('EnsembleModels.csv')
ensemlbesResults

RMSE  MAPE   MAE  Differencing  \
Model             Features                                                    
AdaBoostRegressor Covid data as exog        84.52  0.39 52.17          True   
                  Covid data as exog        44.98  0.28 28.52         False   
                  No exogenous              91.89  0.40 57.92          True   
                  No exogenous              44.65  0.28 27.87         False   
                  Quarantines as exogenous  93.31  0.37 56.77          True   
                  Quarantines as exogenous  45.70  0.29 29.38         False   
                  Time as exogenous         87.34  0.39 53.97          True   
                  Time as exogenous         44.94  0.28 28.15         False   
HGBRegressor      Covid data as exog        93.66  0.44 59.23          True   
                  Covid data as exog        45.55  0.30 30.32         False   
                  No exogenous              83.55  0.49 55.73          True   
                  No exogenous              44.19  0.29 28.73         False   
                  Quarantines as exogenous  80.80  0.58 58.73          True   
                  Quarantines as exogenous  45.02  0.30 29.23         False   
                  Time as exogenous        104.46  0.60 75.22          True   
                  Time as exogenous         43.56  0.29 28.67         False   
XGBoost           Covid data as exog        86.19  0.48 57.20          True   
                  Covid data as exog        51.81  0.31 36.60         False   
                  No exogenous             108.67  0.67 80.25          True   
                  No exogenous              46.52  0.31 30.65         False   
                  Quarantines as exogenous 101.67  0.65 73.93          True   
                  Quarantines as exogenous  47.41  0.33 32.30         False   
                  Time as exogenous        108.82  0.64 78.72          True   
                  Time as exogenous         45.76  0.32 32.43         False   

                                                   Time  
Model             Features                               
AdaBoostRegressor Covid data as exog        0:00:00.205  
                  Covid data as exog        0:00:00.220  
                  No exogenous              0:00:00.151  
                  No exogenous              0:00:00.160  
                  Quarantines as exogenous  0:00:00.161  
                  Quarantines as exogenous  0:00:00.171  
                  Time as exogenous         0:00:00.223  
                  Time as exogenous         0:00:00.236  
HGBRegressor      Covid data as exog        0:00:00.553  
                  Covid data as exog        0:00:00.611  
                  No exogenous              0:00:00.465  
                  No exogenous              0:00:00.483  
                  Quarantines as exogenous  0:00:00.537  
                  Quarantines as exogenous  0:00:00.598  
                  Time as exogenous         0:00:00.709  
                  Time as exogenous         0:00:00.755  
XGBoost           Covid data as exog        0:00:00.082  
                  Covid data as exog        0:00:00.083  
                  No exogenous              0:00:00.070  
                  No exogenous              0:00:00.064  
                  Quarantines as exogenous  0:00:00.065  
                  Quarantines as exogenous  0:00:00.071  
                  Time as exogenous         0:00:00.082  
                  Time as exogenous         0:00:00.087